# Latex to Excel
Ce notebook permet de réaliser la conversion des tableaux depuis le code latex jusqu’à un format excel.
Nous mettons le fichier en liste puis string, sur laquelle nous appliquons un pattern regex pour sélectionner les tableaux.
Enfin nous utilisons openpyxl pour transformer les informations en tableur excel.

In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font
import re

Nous mettons le fichier en liste

In [2]:
d = []
with open("../Risks/RiskAnalysis/riskAnalysis.tex") as f:
    for line in f:
        d.append(line.replace('\n', '').replace("\\\\", "\\"))


Nous utilsons le Regex

In [3]:
group = []
regex = r"(begin{tabular}.*?end{tabular})"

test_str = ' '.join(d)

matches = re.finditer(regex, test_str, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        group.append(match.group(groupNum))
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

Match 1 was found at 2188-10944: begin{tabular}{p{2cm}p{6,1cm}p{1cm}p{1cm}p{12.25cm}} \hline  	\cellcolor{violet!50}& \cellcolor{violet!50}& \cellcolor{violet!50}& \cellcolor{violet!50}&\bf Périmètre \cellcolor{violet!50}\ \cellcolor{violet!50} & \bf Le périmètre projet a-t-il été établi avec une précision suffisant ? & \bf OUI \cellcolor{green!50}& \bf NON\cellcolor{red!50} & Le projet est une application mobile reconnaissant le style de vêtement porté par un individu et lui recommandant des articles dans le même style. \ DIMENSION 1\cellcolor{violet!50} & \bf Le volume du projet est-il important ? & \bf OUI \cellcolor{red!50}& \bf NON\cellcolor{green!50} & Le projet est un MVP pour un COMEX, il pourrait grossir s’il attire beaucoup d’investisseur \ \cellcolor{violet!50} & \bf Le projet aborde-t-il plusieurs domaines, implique-i-il plusieurs réalisations différentes ? & \bf OUI \cellcolor{green!50}& \bf NON\cellcolor{red!50} & Le projet contient une partie web et gestion des accès, un

Nous obtenons 11 tableaux

In [4]:
len(group)

11

# Tableau Périmètre fonctionnel

In [5]:
data = group[9]

In [6]:
data = re.findall(r'\\hline.*\\hline', data)[0]

In [7]:
data[:200]

'\\hline  Ref.\t\\cellcolor{gray!25}&Type \\cellcolor{gray!25}& Nom\\cellcolor{gray!25}& Descriptif \\cellcolor{gray!25}& \\bf Estimation \\cellcolor{gray!25} & \\bf Complexité \\cellcolor{gray!25} & \\bf Priorit'

Nous extrayons manuellement le tableau du fichier latex.

Nous remplaçons quelques éléments de formatage.

In [8]:
d = data.replace('\\cellcolor{gray!50}', '')\
.replace('\\cellcolor{gray!15}', '')\
.replace('\\cellcolor{gray!25}', '')\
.replace('\\bf', ' ')\
.replace('\\dots', '...')\
.replace('\\hline', '').replace('\n',' ')\
.replace('\t', ' ').replace('\\ ', ' \n ').replace('&', ' \t ')


In [9]:
# Nous sauvegardons dans un format csv
with open('per_funct.csv', 'w') as f:
    f.writelines(d.encode('utf-8').decode())

Nous enregistrons le csv sous forme de tableur excel 2010 "xlsx", afin de pouvoir utiliser la bibliothèque openpyxl.

In [10]:
pd.read_csv('per_funct.csv', sep='\t').to_excel('per_funct.xlsx')

In [11]:
wb = load_workbook(filename = 'per_funct.xlsx')
sheet_ranges = wb['Sheet1']
print(sheet_ranges['B3'].value)

      A1 


Nous créons les couleurs de fond que nous utiliserons dans le fichier

In [12]:
grayLightFill = PatternFill(start_color='e0dede',
                   end_color='e0dede',
                   fill_type='solid')
grayDarkFill = PatternFill(start_color='a7a7a7',
                   end_color='a7a7a7',
                   fill_type='solid')
blackFill = PatternFill(start_color='404040',
                   end_color='404040',
                   fill_type='solid')

Nous appliquons le formatage.

In [13]:
for cell in sheet_ranges['B1:J1']:
    for x in range(len(cell)):
        cell[x].fill = blackFill
        cell[x].font = Font(color="FFFFFF", bold=True) 
n = 2
for cell1 in sheet_ranges['B2:B40']:
    text = str(cell1[0].value).rstrip().lstrip()
    if text.isalpha():
        for cell in sheet_ranges['B' + str(n) + ':J' + str(n)]:
            for x in range(len(cell)):
                cell[x].fill = grayDarkFill
    else:
        for cell in sheet_ranges['B' + str(n) + ':J' + str(n)]:
            for x in range(len(cell)):
                cell[x].fill = grayLightFill
    n += 1


In [14]:
# Code pour détruire la colonne d’index supplémentaire issue du cadre de données.
sheet_ranges.delete_cols(1, 1)
sheet_ranges.title = "Tasks"

In [15]:
wb.save(filename = 'dest_functional_perimeter.xlsx')

# Tableau BackLog
Nous répétons l’opération pour le tableau suivant.

In [16]:
data = group[10]

In [17]:
data = re.findall(r'\\hline.*\\hline', data)[0]

In [18]:
data

"\\hline  Titre\t\\cellcolor{gray!50}&En tant que \\cellcolor{gray!50}& Je veux\\dots\\cellcolor{gray!50}&  \\dots afin de\\dots\\cellcolor{gray!50}& \\bf Pondération\\cellcolor{gray!50} & \\bf Données nécessaires\\cellcolor{gray!50} \\ \\hline  Connexion via une adresse mail&  Utilisateur de l'application &   pouvoir me connecter à mon compte sur l'application mobile en utilisant une adresse mail& je peux voir les informations me concernant (infos personnelles, photos prises, articles suggérés, ...)&    A définir en relatif par rapport aux autres user stories en donnant un chiffre parmi : 1, 3, 5, 8, 13, 20, 50, 100.& {•} adresse mail utilisateur  {•}  mot de passe  \\ \\hline   Conserver le mot de passe haché \\cellcolor{gray!15} & Administrateur de Base de données \\cellcolor{gray!15}& Ne pas conserver le mot de passe en clair \\cellcolor{gray!15}&  Réduire le risque de vol d’identité en cas de piratage  \\cellcolor{gray!15}&  L’utilisation d’un salage et d’une graine lors du hachag

In [19]:
d = data.replace('\\cellcolor{gray!50}', '')\
.replace('\\cellcolor{gray!15}', '')\
.replace('\\cellcolor{gray!25}', '')\
.replace('\\bf', ' ')\
.replace('\\dots', '...')\
.replace('\\hline', '')\
.replace('\t', ' ').replace('\\ ', ' \n ').replace('&', ' \t ')


In [20]:
with open('backlog.csv', 'w') as f:
    f.writelines(d.encode('utf-8').decode())

Nous sauvegardons le fichier csv en tableur excel.

In [21]:
pd.read_csv('backlog.csv', sep='\t').to_excel('backlog.xlsx')

In [22]:
from openpyxl import load_workbook
wb_backlog = load_workbook(filename = 'backlog.xlsx')
ws_backlog = wb_backlog['Sheet1']
print(ws_backlog['B3'].value)

    Conserver le mot de passe haché   


In [23]:
ws_backlog.delete_cols(1, 1)

In [24]:
for cell in ws_backlog['A1:F1']:
    for x in range(len(cell)):
        cell[x].fill = blackFill
        cell[x].font = Font(color="FFFFFF", bold=True) 
n = 2
for cell1 in ws_backlog['A2:A11']:
    text = str(cell1[0].value).rstrip().lstrip()
    if n % 2 != 0:
        for cell in ws_backlog['A' + str(n) + ':F' + str(n)]:
            for x in range(len(cell)):
                cell[x].fill = grayLightFill
    n += 1

In [25]:
ws_backlog.title = "User_stories"

wb_backlog.save(filename = 'dest_backlog.xlsx')

In [26]:
xl1 = load_workbook('dest_functional_perimeter.xlsx')
# sheet you want to copy
s = load_workbook('dest_backlog.xlsx').active
s._parent = xl1
xl1._add_sheet(s)
xl1.save('backlog.xlsx')

Nous obtenons un fichier contenant 2 onglets : le backlog et le périmètre fonctionnel.

# Color Format for Plan d’action contre les risques
Nous répétons l’opération pour le tableau suivant.

In [27]:
data = group[2]
data = re.findall(r'\\hline.*\\hline', data)[0]

In [28]:
d = data.replace('\\cellcolor{gray!50}', '')\
.replace('\\cellcolor{gray!15}', '')\
.replace('\\cellcolor{gray!25}', '')\
.replace('\\cellcolor{red!75}', '')\
.replace('\\cellcolor{red!50}', '')\
.replace('\\cellcolor{yellow!50}', '')\
.replace('\\cellcolor{green!50}', '')\
.replace('\\cellcolor{yellow!75}', '')\
.replace("\\parbox{1.5cm}{", " ")\
.replace("}", " ")\
.replace('\\bf', ' ')\
.replace('\\dots', '...')\
.replace('\\hline', '')\
.replace('\t', ' ').replace('\\ ', ' \n ').replace('&', ' \t ')

In [29]:
with open('risk_prevention.csv', 'w') as f:
    f.writelines(d.encode('utf-8').decode())

In [30]:
pd.read_csv('risk_prevention.csv', sep='\t').to_excel('risk_prevention.xlsx')

In [31]:
from openpyxl import load_workbook
wb_backlog = load_workbook(filename = 'risk_prevention.xlsx')
ws_backlog = wb_backlog['Sheet1']
print(ws_backlog['B3'].value)

    Codeur   


In [32]:
greenLightFill = PatternFill(start_color='80ff80',
                   end_color='80ff80',
                   fill_type='solid')
yellowFill = PatternFill(start_color='fff980',
                   end_color='fff980',
                   fill_type='solid')
redLightFill = PatternFill(start_color='ff8080',
                   end_color='ff8080',
                   fill_type='solid')
redDarkFill = PatternFill(start_color='ff4040',
                   end_color='ff4040',
                   fill_type='solid')

In [33]:
ws_backlog.delete_cols(1, 1)

In [34]:
for l in ['F', 'K']:
    for cell in ws_backlog[ l + '2:' + l + '13']:
        text = str(cell[0].value).rstrip().lstrip()
        for x in range(len(cell)):
            if float(text) <= 5. :
                cell[x].fill = greenLightFill
            elif (float(text) > 5.) & (float(text) <= 10.) :
                cell[x].fill = yellowFill
            elif (float(text) > 10.) & (float(text) <= 15.) :
                cell[x].fill = redLightFill
            elif float(text) > 15. :
                cell[x].fill = redDarkFill

In [35]:
ws_backlog.title = "Risk_prevention"
wb_backlog.save(filename = 'risk_prevention.xlsx')